In [1]:
import os
import requests
from langchain.chat_models import ChatOpenAI
from langchain.agents import Tool, initialize_agent, AgentType
from langchain.memory import ConversationBufferMemory
import warnings
warnings.filterwarnings("ignore")
from dotenv import load_dotenv
from langchain_google_genai import ChatGoogleGenerativeAI

load_dotenv()

GEMINI_API_KEY = os.getenv("GOOGLE_API_KEY")
weather_api_key = os.getenv("weather_api_key")

def get_weather(city):
    url = f"http://api.openweathermap.org/data/2.5/weather?q={city}&appid={weather_api_key}&units=metric"
    response = requests.get(url).json()
    if response.get("cod") != 200:
        return f"Weather data not found for {city}."
    weather = response["weather"][0]["description"]
    temp = response["main"]["temp"]
    return f"The current weather in {city} is {weather} with a temperature of {temp}°C."

# Define weather tool
weather_tool = Tool(
    name="weather",
    func=lambda city: get_weather(city),
    description="Provides current weather information for a given city."
)



# Set up memory for personalized conversations
memory = ConversationBufferMemory()

llm = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash",
)

# Initialize the agent with tools and memory
agent = initialize_agent(
    tools=[weather_tool],  # Add more tools if needed (e.g., flight, hotel, etc.)
    llm=llm,
    agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    memory=memory,
    verbose=True
)

# Example user interactions
# 1. User asks for a sunny destination
user_query = input("Enter destination location : ")
response_1 = agent.run(user_query)
print(response_1)




c:\Users\DELL\AppData\Local\Programs\Python\Python310\lib\importlib\__init__.py:126: LangChainDeprecationWarning: As of langchain-core 0.3.0, LangChain uses pydantic v2 internally. The langchain_core.pydantic_v1 module was a compatibility shim for pydantic v1, and should no longer be used. Please update the code to import from Pydantic directly.

For example, replace imports like: `from langchain_core.pydantic_v1 import BaseModel`
with: `from pydantic import BaseModel`
or the v1 compatibility namespace if you are working in a code base that has not been fully upgraded to pydantic 2 yet. 	from pydantic.v1 import BaseModel

  return _bootstrap._gcd_import(name[level:], package, level)
c:\Users\DELL\AppData\Local\Programs\Python\Python310\lib\site-packages\pydantic\_internal\_config.py:341: UserWarning: Valid config keys have changed in V2:
* 'allow_population_by_field_name' has been renamed to 'populate_by_name'
  warnings.warn(message, UserWarning)




> Entering new AgentExecutor chain...
Action: weather
Action Input: Dubai
Observation: The current weather in Dubai is broken clouds with a temperature of 42.49°C.
Thought:I now know the final answer
Final Answer: The current weather in Dubai is broken clouds with a temperature of 42.49°C.

> Finished chain.
The current weather in Dubai is broken clouds with a temperature of 42.49°C.
